In [24]:
from urllib2 import urlopen
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

from bs4 import BeautifulSoup
import re
import json

In [28]:
def startWebDriver():
    global driver
    options = Options()
    options.add_argument("--disable-extensions")
    driver = webdriver.Chrome(executable_path = '/home/abhishek/Downloads/chromedriver_linux64/chromedriver',options=options)
    
def getJSONObject(Link):
    HTML = urlopen(Link)
    soup = BeautifulSoup(HTML,'lxml')
    
    text = []
    
    for para in soup.find_all("p"):
        paragraph = str(para.text)
        text.append(paragraph)
        
    text = text[6:9]
    
    JSONObject = {}
    #empty_JSONObject = {}
    
    bad_report_flag = 0
    field_count = 0
    
    for item in text:
        m = re.search('(.*): (.*)',item)
        
        # @ - start of sequence tag
        # $ - end of sequence tag
        if m and m.group(1) in ['Impression','Findings','Indication']:
            field_count = field_count + 1
            JSONObject[m.group(1)] = m.group(2)
        else:
            bad_report_flag = 1
            break;
    
    if field_count == 3 and bad_report_flag == 0:
        return JSONObject 
    else:
        return None

In [ ]:
startWebDriver()
driver.get('https://openi.nlm.nih.gov/gridquery.php?q=&coll=cxr')
time.sleep(4) #set wait time

element = driver.find_element_by_xpath('//*[@id="imagearea"]/div[1]/div/a/img').click()

time.sleep(4)

with open('reports_all.jsonl', 'w') as file:
    
    #loop for 7469 reports 
    for i in range(0,1000):
        print(i)
        #get current url
        link = driver.current_url
        JSONObject = getJSONObject(link)
        
        if(JSONObject):
            json.dump(JSONObject , file, ensure_ascii=False)
        
        # click on the > button
        element = driver.find_element_by_xpath('//*[@id="imageClassM"]/div/a[2]/img').click()
        time.sleep(2)